In [1]:
"""
Autohr: Sergi Mas Pujol
Last update: 16/12/2020

Python version: 3.6
"""

'\nAutohr: Sergi Mas Pujol\nLast update: 16/12/2020\n\nPython version: 3.6\n'

In [2]:
import numpy as np
np.random.seed(4)
import sys
import math
import random
random.seed(7)
from random import sample
from datetime import datetime, timedelta, date

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 7]

from utils_samplesTraining import read_REGULATIONS_file
from utils_timeProcessing import from_YYYYMMDDHHMMSS_to_HHMMSS_withTwoDots, \
                                 substract_minutes_given_HHSSMM, add_minutes_given_HHSSMM, \
                                 from_YYYYMMDD_to_DDMMYYYY_given_separator, \
                                 listDays_betweenTwoDates, \
                                 from_YYYYMMDDHHMMSS_to_HHMMSS
from utils_samplesTraining import readAssociatedFile_fromAIRAC_givenDate, \
                                  extract_regulations,\
                                  addIntervals_toFinalConjunt
from generator import extract_days_timestamps_volumes_labels_days_with_regulations, \
                      extract_features_from_list_days_and_timestamps

from generator_weather import extract_weather_information_from_list_days_and_timestamps

In [3]:
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, TimeDistributed

from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/home/sergi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sergi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sergi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sergi/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [4]:
# sectorName = 'BOLN'
# sectorName = 'D6WH'
# sectorName = 'B3EH'

sectorName = 'HRHR'
# sectorName = 'HSOL'
# sectorName = 'B3LL'

# sectorName = 'LFEEHR'

# MASD3WLC

In [5]:
# Given the start timestamp of the regulation -> Considered time before and after
# Temporal gaps in minutes
gap_before_start_time = 0
gap_after_start_time = 30

In [6]:
num_epochs = 200
batch_size = 32

# Used to extract more samples from a particular sector
if sectorName == 'BOLN':
    num_additional_samples_per_day = 25
else:
    num_additional_samples_per_day = 75

In [7]:
# num_weather_features = 7
num_weather_features = 16
num_metric_per_weather_feature = 3

# Prepare / Extreact data

###  Extract the available regulations

In [8]:
REGULATIONS = read_REGULATIONS_file('./20190604_20191020_REGULATIONS.csv')

REGULATIONS_from_given_sector = REGULATIONS.loc[(REGULATIONS["traffic_volume"] == ' MAS'+sectorName+' ') &
                                                (REGULATIONS["regulation_reason"].isin([' W-Weather '])) & 
                                                (REGULATIONS["location_type"] == ' En route ') &
                                                (REGULATIONS["cancel_time"] == '  ')
                                               ]



In [9]:
# ".strip() -> Remove all the white spaaces in the timestamp"

REGULATIONS_from_given_sector.loc[:, 'date_DDMMYYYY'] = REGULATIONS_from_given_sector.apply(lambda x: from_YYYYMMDD_to_DDMMYYYY_given_separator(x.target_date.strip(), '/'), axis=1)

REGULATIONS_from_given_sector.loc[:, 'start_time_HHMMSS'] = REGULATIONS_from_given_sector.apply(lambda x: from_YYYYMMDDHHMMSS_to_HHMMSS_withTwoDots(x.start_time.strip()), axis=1)
REGULATIONS_from_given_sector.loc[:, 'end_time_HHMMSS'] = REGULATIONS_from_given_sector.apply(lambda x: from_YYYYMMDDHHMMSS_to_HHMMSS_withTwoDots(x.end_time.strip()), axis=1)
REGULATIONS_from_given_sector.loc[:, 'start_time_study_HHMMSS'] = REGULATIONS_from_given_sector.apply(lambda x: substract_minutes_given_HHSSMM(x.start_time_HHMMSS, gap_before_start_time), axis=1)
REGULATIONS_from_given_sector.loc[:, 'end_time_study_HHMMSS'] = REGULATIONS_from_given_sector.apply(lambda x: add_minutes_given_HHSSMM(x.start_time_HHMMSS, gap_after_start_time), axis=1)

/home/sergi/.local/lib/python3.6/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/sergi/.local/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [10]:
# We want to remove from the DataFrame all the rows outside the valid interval of days
# Outside the date of the available AIRACS
min_date = date(2019, 6, 4)
max_date = date(2019, 9, 11) 


for index, row in REGULATIONS_from_given_sector.iterrows():
        
    date_split = row["date_DDMMYYYY"].split("/") # 0 -> day; 1 -> month; 2 -> year
    date_object = date(int(date_split[2]), int(date_split[1]), int(date_split[0]))
    
    # Check if the regulation's date belong to the valid interval -> IF outside, drop the Regulation
    if date_object < min_date or date_object > max_date:
        REGULATIONS_from_given_sector = REGULATIONS_from_given_sector.drop(index=index)

In [11]:
print('Number regulations AFTER pre-processing: ', str(len(REGULATIONS_from_given_sector.values)))

Number regulations AFTER pre-processing:  15


### Extract the different list of days and timestamps

##### Samples without regulations from days without regulations

In [12]:
# Extract the days with regulations
list_days_with_regulations = REGULATIONS_from_given_sector["date_DDMMYYYY"].values

# Initialize a list with all the possible days and the final list for the days without regulations
list_all_possible_days = listDays_betweenTwoDates(min_date, max_date, 'DDMMYYYY', '/')
list_days_without_regulations = list()

In [13]:
# If a day does not appear in the list with reguilations -> No regulations for that day
for day in list_all_possible_days:
    if day not in list_days_with_regulations:
        list_days_without_regulations.append(day)

In [14]:
# For the days without regulations, we will generate random timestamps

list_days_without_regulations_extended = list()
start_time_samples_no_regulations = list()
end_time_samples_no_regulations = list()

for day in list_days_without_regulations:
    # Iterate multiple time each day without regulations
    for _ in range(0,1):
        # Randomly create the hour and the minutes
        hour = str("{:02d}".format(random.randint(3,21))) # From 2 to 22 to avoid extrems (wrap around timestamps)
        minute = str("{:02d}".format(random.randint(0,50)))

        # Create the timestamp
        random_timestamps = hour + ":" + minute + ":" + '00'

        # Append the day of the timestamp
        list_days_without_regulations_extended.append(day)
        
        # Substract a given about to create the starting timestamp
        start_timestamp = substract_minutes_given_HHSSMM(random_timestamps, gap_before_start_time)
        start_time_samples_no_regulations.append(start_timestamp)

        # Add a given about to create the ending timestamp
        end_timestamp = add_minutes_given_HHSSMM(random_timestamps, gap_after_start_time)
        end_time_samples_no_regulations.append(end_timestamp)

In [15]:
len(list_days_without_regulations_extended), len(start_time_samples_no_regulations), \
len(end_time_samples_no_regulations)

(88, 88, 88)

##### Sample with & without regulations from days with regulations completly random 
##### It is possible to have sample with both categories (e.g. start NO Regs from Regs and then Reg)

In [19]:
REGULATIONS_from_given_sector.loc[:, 'start_regulations_HHMMSS'] = REGULATIONS_from_given_sector.apply(lambda x: from_YYYYMMDDHHMMSS_to_HHMMSS(x.start_time.strip()), axis=1)
REGULATIONS_from_given_sector.loc[:, 'end_regulations_HHMMSS'] = REGULATIONS_from_given_sector.apply(lambda x: from_YYYYMMDDHHMMSS_to_HHMMSS(x.end_time.strip()), axis=1)

In [20]:
list_dates_with_regulations = REGULATIONS_from_given_sector["date_DDMMYYYY"].values

start_regulations = REGULATIONS_from_given_sector["start_regulations_HHMMSS"].values
end_regulations = REGULATIONS_from_given_sector["end_regulations_HHMMSS"].values

list_volumes_regulations = REGULATIONS_from_given_sector["traffic_volume"].values

In [26]:
list_days_from_regulations, \
start_time_samples_days_from_regulations, \
end_time_samples_days_from_regulations, \
volumes_days_from_regulations, \
labels_days_from_regulations = extract_days_timestamps_volumes_labels_days_with_regulations(
                                          list_dates_with_regulations, 
                                          start_regulations,
                                          end_regulations,
                                          list_volumes_regulations, 
                                          gap_before_start_time, 
                                          gap_after_start_time, 
                                          num_additional_samples_per_day=num_additional_samples_per_day,
                                          min_timestamps_with_regulations=10)

In [27]:
len(list_days_from_regulations), len(start_time_samples_days_from_regulations), \
len(end_time_samples_days_from_regulations), len(volumes_days_from_regulations)

(387, 387, 387, 387)

# Extract all the features for the given dates and timestamps

In [33]:
list_days_without_regulations_extended = np.array(list_days_without_regulations_extended)
start_time_samples_no_regulations = np.array(start_time_samples_no_regulations)
end_time_samples_no_regulations = np.array(end_time_samples_no_regulations)

In [34]:
X_days_without_regulations = np.zeros((len(list_days_without_regulations_extended), 
                                       int(gap_before_start_time+gap_after_start_time), 
                                       11))
X_days_without_regulations.shape

(88, 30, 11)

In [35]:
X_days_without_regulations = extract_features_from_list_days_and_timestamps(list_days_without_regulations_extended, 
                                                                           start_time_samples_no_regulations, 
                                                                           end_time_samples_no_regulations,
                                                                           sectorName,
                                                                           gap_before_start_time, gap_after_start_time,
                                                                           X_days_without_regulations)

HRHR | counter: 1
HRHR | counter: 2
HRHR | counter: 3
HRHR | counter: 4
HRHR | counter: 5
HRHR | counter: 6
HRHR | counter: 7
HRHR | counter: 8
HRHR | counter: 9
HRHR | counter: 10
HRHR | counter: 11
HRHR | counter: 12
HRHR | counter: 13
HRHR | counter: 14
HRHR | counter: 15
HRHR | counter: 16
HRHR | counter: 17
HRHR | counter: 18
HRHR | counter: 19
HRHR | counter: 20
HRHR | counter: 21
HRHR | counter: 22
HRHR | counter: 23
HRHR | counter: 24
HRHR | counter: 25
HRHR | counter: 26
HRHR | counter: 27
HRHR | counter: 28
HRHR | counter: 29
HRHR | counter: 30
HRHR | counter: 31
HRHR | counter: 32
HRHR | counter: 33
HRHR | counter: 34
HRHR | counter: 35
HRHR | counter: 36
HRHR | counter: 37
HRHR | counter: 38
HRHR | counter: 39
HRHR | counter: 40
HRHR | counter: 41
HRHR | counter: 42
HRHR | counter: 43
HRHR | counter: 44
HRHR | counter: 45
HRHR | counter: 46
HRHR | counter: 47
HRHR | counter: 48
HRHR | counter: 49
HRHR | counter: 50
HRHR | counter: 51
HRHR | counter: 52
HRHR | counter: 53
HR

In [23]:
np.save('./Counting_variables/Weather/X_days_without_regulations', X_days_without_regulations)

In [24]:
list_days_from_regulations = np.array(list_days_from_regulations)
start_time_samples_days_from_regulations = np.array(start_time_samples_days_from_regulations)
end_time_samples_days_from_regulations = np.array(end_time_samples_days_from_regulations)

In [25]:
X_days_from_regulations = np.zeros((len(list_days_from_regulations), 
                                    int(gap_before_start_time+gap_after_start_time), 
                                    11))
X_days_from_regulations.shape

(246, 30, 11)

In [26]:
X_days_from_regulations = extract_features_from_list_days_and_timestamps(list_days_from_regulations, 
                                                                       start_time_samples_days_from_regulations, 
                                                                       end_time_samples_days_from_regulations,
                                                                       sectorName,
                                                                       gap_before_start_time, gap_after_start_time,
                                                                       X_days_from_regulations)

HSOL | counter: 1
HSOL | counter: 2
HSOL | counter: 3
HSOL | counter: 4
HSOL | counter: 5
HSOL | counter: 6
HSOL | counter: 7
HSOL | counter: 8
HSOL | counter: 9
HSOL | counter: 10
HSOL | counter: 11
HSOL | counter: 12
HSOL | counter: 13
HSOL | counter: 14
HSOL | counter: 15
HSOL | counter: 16
HSOL | counter: 17
HSOL | counter: 18
HSOL | counter: 19
HSOL | counter: 20
HSOL | counter: 21
HSOL | counter: 22
HSOL | counter: 23
HSOL | counter: 24
HSOL | counter: 25
HSOL | counter: 26
HSOL | counter: 27
HSOL | counter: 28
HSOL | counter: 29
HSOL | counter: 30
HSOL | counter: 31
HSOL | counter: 32
HSOL | counter: 33
HSOL | counter: 34
HSOL | counter: 35
HSOL | counter: 36
HSOL | counter: 37
HSOL | counter: 38
HSOL | counter: 39
HSOL | counter: 40
HSOL | counter: 41
HSOL | counter: 42
HSOL | counter: 43
HSOL | counter: 44
HSOL | counter: 45
HSOL | counter: 46
HSOL | counter: 47
HSOL | counter: 48
HSOL | counter: 49
HSOL | counter: 50
HSOL | counter: 51
HSOL | counter: 52
HSOL | counter: 53
HS

In [27]:
np.save('./Counting_variables/Weather/X_days_from_regulations', X_days_from_regulations)
np.save('./Counting_variables/Weather/labels_days_from_regulations', labels_days_from_regulations)

# Extract the weather features 

In [28]:
X_days_without_regulations_weather = np.zeros((len(list_days_without_regulations_extended), 
                                               int(gap_before_start_time+gap_after_start_time), 
                                               num_weather_features*num_metric_per_weather_feature))
X_days_without_regulations_weather.shape

(89, 30, 48)

In [29]:
X_days_without_regulations_weather = extract_weather_information_from_list_days_and_timestamps(
    list_days_without_regulations_extended, 
    start_time_samples_no_regulations, 
    end_time_samples_no_regulations,
    sectorName,
    num_weather_features,
    num_metric_per_weather_feature,
    X_days_without_regulations_weather)

day: 04/06/2019 | start:130900 | TV: HSOL
day: 05/06/2019 | start:154100 | TV: HSOL
day: 06/06/2019 | start:040400 | TV: HSOL
day: 07/06/2019 | start:200600 | TV: HSOL
day: 08/06/2019 | start:143700 | TV: HSOL
day: 09/06/2019 | start:043200 | TV: HSOL
day: 14/06/2019 | start:090200 | TV: HSOL
day: 15/06/2019 | start:052700 | TV: HSOL
day: 16/06/2019 | start:160400 | TV: HSOL
day: 17/06/2019 | start:100500 | TV: HSOL
day: 18/06/2019 | start:202700 | TV: HSOL
day: 21/06/2019 | start:043600 | TV: HSOL
day: 22/06/2019 | start:061400 | TV: HSOL
day: 23/06/2019 | start:210300 | TV: HSOL
day: 24/06/2019 | start:213700 | TV: HSOL
day: 25/06/2019 | start:150300 | TV: HSOL
day: 26/06/2019 | start:100200 | TV: HSOL
day: 27/06/2019 | start:200800 | TV: HSOL
day: 28/06/2019 | start:122600 | TV: HSOL
day: 29/06/2019 | start:073400 | TV: HSOL
day: 30/06/2019 | start:063600 | TV: HSOL
day: 01/07/2019 | start:123500 | TV: HSOL
day: 02/07/2019 | start:080600 | TV: HSOL
day: 03/07/2019 | start:213600 | T

In [30]:
np.save('./Counting_variables/Weather/X_days_without_regulations_weather', X_days_without_regulations_weather)

In [31]:
X_days_from_regulations_weather = np.zeros((len(list_days_from_regulations), 
                                            int(gap_before_start_time+gap_after_start_time), 
                                            num_weather_features*num_metric_per_weather_feature))
X_days_from_regu<lations_weather.shape

(246, 30, 48)

In [32]:
X_days_from_regulations_weather = extract_weather_information_from_list_days_and_timestamps(
    list_days_from_regulations, 
    start_time_samples_days_from_regulations, 
    end_time_samples_days_from_regulations,
    sectorName,
    num_weather_features,
    num_metric_per_weather_feature,
    X_days_from_regulations_weather)

day: 10/06/2019 | start:174000 | TV: HSOL
day: 10/06/2019 | start:182900 | TV: HSOL
day: 10/06/2019 | start:193300 | TV: HSOL
day: 10/06/2019 | start:074400 | TV: HSOL
day: 10/06/2019 | start:192900 | TV: HSOL
day: 10/06/2019 | start:141100 | TV: HSOL
day: 10/06/2019 | start:034000 | TV: HSOL
day: 10/06/2019 | start:094000 | TV: HSOL
day: 10/06/2019 | start:165000 | TV: HSOL
day: 10/06/2019 | start:171900 | TV: HSOL
day: 10/06/2019 | start:183200 | TV: HSOL
day: 10/06/2019 | start:073700 | TV: HSOL
day: 10/06/2019 | start:193600 | TV: HSOL
day: 10/06/2019 | start:191800 | TV: HSOL
day: 10/06/2019 | start:184900 | TV: HSOL
day: 10/06/2019 | start:194300 | TV: HSOL
day: 10/06/2019 | start:194800 | TV: HSOL
day: 10/06/2019 | start:192200 | TV: HSOL
day: 10/06/2019 | start:184200 | TV: HSOL
day: 10/06/2019 | start:104300 | TV: HSOL
day: 10/06/2019 | start:114500 | TV: HSOL
day: 11/06/2019 | start:063500 | TV: HSOL
day: 11/06/2019 | start:124200 | TV: HSOL
day: 11/06/2019 | start:062400 | T

day: 09/08/2019 | start:172600 | TV: HSOL
day: 09/08/2019 | start:133300 | TV: HSOL
day: 09/08/2019 | start:174000 | TV: HSOL
day: 09/08/2019 | start:081900 | TV: HSOL
day: 09/08/2019 | start:063300 | TV: HSOL
day: 09/08/2019 | start:183400 | TV: HSOL
day: 09/08/2019 | start:172700 | TV: HSOL
day: 09/08/2019 | start:044900 | TV: HSOL
day: 09/08/2019 | start:211900 | TV: HSOL
day: 09/08/2019 | start:183200 | TV: HSOL
day: 09/08/2019 | start:201500 | TV: HSOL
day: 09/08/2019 | start:204500 | TV: HSOL
day: 09/08/2019 | start:183400 | TV: HSOL
day: 09/08/2019 | start:192600 | TV: HSOL
day: 09/08/2019 | start:193000 | TV: HSOL
day: 09/08/2019 | start:184200 | TV: HSOL
day: 09/08/2019 | start:191900 | TV: HSOL
day: 09/08/2019 | start:101200 | TV: HSOL
day: 09/08/2019 | start:183300 | TV: HSOL
day: 09/08/2019 | start:092600 | TV: HSOL
day: 18/08/2019 | start:174700 | TV: HSOL
day: 18/08/2019 | start:132200 | TV: HSOL
day: 18/08/2019 | start:174100 | TV: HSOL
day: 18/08/2019 | start:083300 | T

In [33]:
np.save('./Counting_variables/Weather/X_days_from_regulations_weather', X_days_from_regulations_weather)

# Concatenate the scalar variables and the weather features

In [10]:
X_days_without_regulations = np.load('./Counting_variables/Weather/X_days_without_regulations.npy')
X_days_without_regulations_weather = np.load('./Counting_variables/Weather/X_days_without_regulations_weather.npy')

X_days_from_regulations = np.load('./Counting_variables/Weather/X_days_from_regulations.npy')
X_days_from_regulations_weather = np.load('./Counting_variables/Weather/X_days_from_regulations_weather.npy')

labels_days_from_regulations = np.load('./Counting_variables/Weather/labels_days_from_regulations.npy')

In [11]:
X_days_without_regulations_concatenate = np.zeros((X_days_without_regulations.shape[0], 
                                                   X_days_without_regulations.shape[1],
                                                   X_days_without_regulations.shape[2]+X_days_without_regulations_weather.shape[2]))

# X_days_without_regulations_concatenate = np.zeros((X_days_without_regulations.shape[0], 
#                                                    X_days_without_regulations.shape[1],
#                                                    X_days_without_regulations.shape[2]))

X_days_without_regulations_concatenate.shape

(88, 30, 59)

In [12]:
X_days_without_regulations_concatenate[:,:,0:num_weather_features*num_metric_per_weather_feature] = X_days_without_regulations_weather
X_days_without_regulations_concatenate[:,:,num_weather_features*num_metric_per_weather_feature:] = X_days_without_regulations

# X_days_without_regulations_concatenate = X_days_without_regulations

In [13]:
X_days_from_regulations_concatenate = np.zeros((X_days_from_regulations.shape[0], 
                                                X_days_from_regulations.shape[1],
                                                X_days_from_regulations.shape[2]+X_days_from_regulations_weather.shape[2]))

# X_days_from_regulations_concatenate = np.zeros((X_days_from_regulations.shape[0], 
#                                                 X_days_from_regulations.shape[1],
#                                                 X_days_from_regulations.shape[2]))


X_days_from_regulations_concatenate.shape

(387, 30, 59)

In [14]:
X_days_from_regulations_concatenate[:,:,0:num_weather_features*num_metric_per_weather_feature] = X_days_from_regulations_weather
X_days_from_regulations_concatenate[:,:,num_weather_features*num_metric_per_weather_feature:] = X_days_from_regulations

# X_days_from_regulations_concatenate = X_days_from_regulations

# Create the labels

In [15]:
Y_days_without_regulations = np.zeros((X_days_without_regulations.shape[0], (gap_before_start_time+gap_after_start_time), 1), dtype=int)

In [16]:
Y_days_without_regulations.shape

(88, 30, 1)

In [17]:
Y_from_regulations = np.zeros((X_days_from_regulations.shape[0], (gap_before_start_time+gap_after_start_time), 1), dtype=int)

In [18]:
Y_from_regulations = labels_days_from_regulations

In [19]:
Y_from_regulations.shape

(387, 30, 1)

# Split into training & testing - By day

In [20]:
X_days_without_regulations_concatenate.shape, Y_days_without_regulations.shape

((88, 30, 59), (88, 30, 1))

In [21]:
X_days_from_regulations_concatenate.shape, Y_from_regulations.shape

((387, 30, 59), (387, 30, 1))

In [22]:
num_train_samples_days_without_regulations = int(np.floor(X_days_without_regulations_concatenate.shape[0]*0.7))
num_train_samples_days_without_regulations

61

In [23]:
num_train_samples_days_from_regulations = int(np.floor(X_days_from_regulations_concatenate.shape[0]*0.7))
num_train_samples_days_from_regulations

270

In [24]:
X_train = np.concatenate((X_days_without_regulations_concatenate[0:num_train_samples_days_without_regulations],
                          X_days_from_regulations_concatenate[0:num_train_samples_days_from_regulations]))

y_train = np.concatenate((Y_days_without_regulations[0:num_train_samples_days_without_regulations],
                          Y_from_regulations[0:num_train_samples_days_from_regulations]))

In [25]:
X_train.shape, y_train.shape

((331, 30, 59), (331, 30, 1))

In [26]:
X_test = np.concatenate((X_days_without_regulations_concatenate[num_train_samples_days_without_regulations:],
                         X_days_from_regulations_concatenate[num_train_samples_days_from_regulations:]))

y_test = np.concatenate((Y_days_without_regulations[num_train_samples_days_without_regulations:],
                         Y_from_regulations[num_train_samples_days_from_regulations:]))

In [27]:
X_test.shape, y_test.shape

((144, 30, 59), (144, 30, 1))

# Only using the max values form the weather features

In [28]:
X_train.shape, X_test.shape

((331, 30, 59), (144, 30, 59))

In [29]:
list_weather_features_delete = []
for i in range(num_weather_features):
    list_weather_features_delete.append(0+i*num_metric_per_weather_feature)
    list_weather_features_delete.append(1+i*num_metric_per_weather_feature)
    
# Additional elements added to the delete list
list_weather_features_delete.append(1*3-1)
list_weather_features_delete.append(3*3-1)
# list_weather_features_delete.append(4*3-1)
# list_weather_features_delete.append(11*3-1)
# list_weather_features_delete.append(3*3-1)
# list_weather_features_delete.append(3*3-1)

In [30]:
len(list_weather_features_delete)

34

In [31]:
X_train_del = np.zeros((X_train.shape[0], X_train.shape[1], X_train.shape[2]-len(list_weather_features_delete)))
X_test_del = np.zeros((X_test.shape[0], X_test.shape[1], X_train.shape[2]-len(list_weather_features_delete)))

for j in range(X_train.shape[0]):
    X_train_del[j] = np.delete(X_train[j], list_weather_features_delete, axis=1)
for j in range(X_test.shape[0]):
    X_test_del[j] = np.delete(X_test[j], list_weather_features_delete, axis=1)


In [32]:
X_train_del.shape, X_test_del.shape

((331, 30, 25), (144, 30, 25))

In [33]:
# Used it to avoid having to change the name of the variables in the following lines
X_train = X_train_del
X_test = X_test_del

# Using StandardScaler from sklearn.preprocessing

In [34]:
from sklearn.preprocessing import StandardScaler

In [35]:
X_train.shape

(331, 30, 25)

In [36]:
X_train_flatten = np.reshape(X_train, (X_train.shape[0]*X_train.shape[1], X_train.shape[2]))
X_test_flatten = np.reshape(X_test, (X_test.shape[0]*X_test.shape[1], X_train.shape[2]))

# X_train_flatten = np.reshape(X_train, (X_train.shape[0]*X_train.shape[1], 59))
# X_test_flatten = np.reshape(X_test, (X_test.shape[0]*X_test.shape[1], 59))

In [37]:
X_train_flatten.shape, X_test_flatten.shape

((9930, 25), (4320, 25))

In [38]:
scaler = StandardScaler()
scaler.fit(X_train_flatten)
X_sc_train = scaler.transform(X_train_flatten)
X_sc_test = scaler.transform(X_test_flatten)

In [39]:
X_train_sc = np.reshape(X_train, (X_train.shape))
X_test_sc = np.reshape(X_test, (X_test.shape))

In [40]:
X_train_sc.shape, X_test_sc.shape

((331, 30, 25), (144, 30, 25))

In [41]:
# X_sc_train = np.zeros((X_train.shape))
# X_sc_test = np.zeros((X_test.shape))

In [42]:
# # Fit and store a scaler for each channel

# scalers = {}
# for i in range(X_train.shape[1]):
#     scalers[i] = StandardScaler()
#     X_sc_train[:, i, :] = scalers[i].fit_transform(X_train[:, i, :]) 

# for i in range(X_test.shape[1]):
#     X_sc_test[:, i, :] = scalers[i].transform(X_test[:, i, :])

# Train the model

In [43]:
from keras.layers import BatchNormalization
from keras.models import Sequential

In [44]:
# def Hotspoter_v2(input_shape):

#     daily_traffic = Input(shape=input_shape)
    
#     # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
#     # Be careful, the returned output should be a batch of sequences.
#     X = LSTM(59, return_sequences=True)(daily_traffic)
#     X = Dropout(0.5)(X)
# #     X =  BatchNormalization()(X)
    
#     # Propagate X trough another LSTM layer with 128-dimensional hidden state
#     # Be careful, the returned output should be a single hidden state, not a batch of sequences.
#     X = LSTM(64, return_sequences = True)(X)
#     X = Dropout(0.5)(X)
# #     X =  BatchNormalization()(X)
    
#     # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
#     X = TimeDistributed(Dense(1, activation = "sigmoid"))(X) # time distributed  (sigmoid)
    
#     # Create Model instance which converts sentence_indices into X.
#     model = Model(inputs=daily_traffic, outputs=X)

    
#     return model

In [145]:
def Hotspoter_v2(input_shape):

    daily_traffic = Input(shape=input_shape)
    
    X = LSTM(X_train.shape[2], return_sequences=True)(daily_traffic)
    X = Dropout(0.5)(X)
#     X =  BatchNormalization()(X)

    X = LSTM(X_train.shape[2]*2, return_sequences = True)(X)
    X = Dropout(0.5)(X)
#     X =  BatchNormalization()(X)
    
#     X = LSTM(X_train.shape[2]*3, return_sequences = True)(X)
#     X = Dropout(0.5)(X)
#     X =  BatchNormalization()(X)
    
#     X = LSTM(X_train.shape[2], return_sequences = True)(X)
#     X = Dropout(0.5)(X)
#     X =  BatchNormalization()(X)

    X = LSTM(10, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    
    X = TimeDistributed(Dense(1, activation = "sigmoid"))(X) # time distributed  (sigmoid)
    
    model = Model(inputs=daily_traffic, outputs=X)

    
    return model

In [146]:
num_timestamps_per_sample = gap_before_start_time + gap_after_start_time

model = Hotspoter_v2((num_timestamps_per_sample, X_train.shape[2]))
# model = Hotspoter_v2([num_timestamps_per_sample, num_weather_features*num_metric_per_weather_feature+11])

model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 30, 25)            0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 30, 25)            5100      
_________________________________________________________________
dropout_14 (Dropout)         (None, 30, 25)            0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 30, 50)            15200     
_________________________________________________________________
dropout_15 (Dropout)         (None, 30, 50)            0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 30, 10)            2440      
_________________________________________________________________
dropout_16 (Dropout)         (None, 30, 10)            0   

In [147]:
# Anopther option: sparse_categorical_crossentropy
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, 
                    validation_data = (X_test, y_test),
                    epochs = 200, batch_size = batch_size, shuffle=True)

# history = model.fit(X_train_sc, y_train, 
#                     validation_data = (X_test_sc, y_test),
#                     epochs = 300, batch_size = batch_size, shuffle=True)

Train on 331 samples, validate on 144 samples
Epoch 1/200
331/331 [==============================] - 1s 4ms/step - loss: 0.6995 - accuracy: 0.5056 - val_loss: 0.6888 - val_accuracy: 0.5556
Epoch 2/200
331/331 [==============================] - 0s 847us/step - loss: 0.6933 - accuracy: 0.5231 - val_loss: 0.6857 - val_accuracy: 0.5465
Epoch 3/200
331/331 [==============================] - 0s 809us/step - loss: 0.6912 - accuracy: 0.5404 - val_loss: 0.6836 - val_accuracy: 0.5472
Epoch 4/200
331/331 [==============================] - 0s 876us/step - loss: 0.6859 - accuracy: 0.5433 - val_loss: 0.6766 - val_accuracy: 0.5664
Epoch 5/200
331/331 [==============================] - 0s 880us/step - loss: 0.6804 - accuracy: 0.5625 - val_loss: 0.6729 - val_accuracy: 0.5657
Epoch 6/200
331/331 [==============================] - 0s 865us/step - loss: 0.6867 - accuracy: 0.5470 - val_loss: 0.6632 - val_accuracy: 0.5988
Epoch 7/200
331/331 [==============================] - 0s 826us/step - loss: 0.6771 - 

Epoch 57/200
331/331 [==============================] - 0s 797us/step - loss: 0.4348 - accuracy: 0.8108 - val_loss: 0.7234 - val_accuracy: 0.7669
Epoch 58/200
331/331 [==============================] - 0s 813us/step - loss: 0.4792 - accuracy: 0.7777 - val_loss: 0.6083 - val_accuracy: 0.7512
Epoch 59/200
331/331 [==============================] - 0s 799us/step - loss: 0.4481 - accuracy: 0.8008 - val_loss: 0.8055 - val_accuracy: 0.7380
Epoch 60/200
331/331 [==============================] - 0s 785us/step - loss: 0.4412 - accuracy: 0.8037 - val_loss: 0.6049 - val_accuracy: 0.7685
Epoch 61/200
331/331 [==============================] - 0s 814us/step - loss: 0.4173 - accuracy: 0.8201 - val_loss: 0.6474 - val_accuracy: 0.7914
Epoch 62/200
331/331 [==============================] - 0s 774us/step - loss: 0.4138 - accuracy: 0.8313 - val_loss: 0.6736 - val_accuracy: 0.7993
Epoch 63/200
331/331 [==============================] - 0s 773us/step - loss: 0.4264 - accuracy: 0.8231 - val_loss: 0.6253 -

In [ ]:
plt.rcParams['figure.figsize'] = [12, 7]

fig, axs = plt.subplots(nrows=1, ncols=2)
axs[0].plot(history.history['loss'], 'r', linewidth=3.0) 
axs[0].plot(history.history['val_loss'], 'b', linewidth=3.0)
axs[0].legend(['train_loss', 'val_loss'], fontsize=10)
axs[0].set_xlabel('Epochs ', fontsize=12)
axs[0].set_ylabel('Loss', fontsize=12)
axs[0].set_ylim(0,10)
axs[0].set_title('Loss', fontsize=12)

axs[1].plot(history.history['accuracy'], 'r', linewidth=3.0) 
axs[1].plot(history.history['val_accuracy'], 'b', linewidth=3.0)
axs[1].legend(['train_accuracy', 'val_accuracy'], fontsize=10)
axs[1].set_xlabel('Epochs ', fontsize=12)
axs[1].set_ylabel('Precision', fontsize=12) 
axs[1].set_ylim(0,1.2)
axs[1].set_title('Accuracy', fontsize=12)
plt.show()

# Evaluate the model

In [112]:
from metrics import confusion_matrix_sequencialOutput, confusion_matrix_sequencial_output_mean_std

### Training samples

In [113]:
TP, FP, TN, FN, conf_matrix = confusion_matrix_sequencialOutput(model, 
                                                                X_train, y_train, 
                                                                (gap_before_start_time + gap_after_start_time))

In [114]:
conf_matrix

array([[4920.,  676.],
       [ 231., 4103.]])

In [115]:
print("Accuracy training: %.2f" % ((TP+TN)/(TP+FP+FN+TN)*100) + "%")
print("Recall training: %.2f" % (TP/(TP+FN)*100) + "%")
print("Precicion training: %.2f" % (TP/(TP+FP)*100) + "%")

Accuracy training: 90.87%
Recall training: 95.52%
Precicion training: 87.92%


In [116]:
acc, acc_std, rec, rec_std, pres, pres_std = confusion_matrix_sequencial_output_mean_std(model,
                                                                                        X_train,
                                                                                        y_train,
                                                                                        (gap_before_start_time + gap_after_start_time),
                                                                                        batch_size*2)

In [117]:
acc, acc_std, rec, rec_std, pres, pres_std

(0.89765625,
 0.03204141098998555,
 0.9564014371717624,
 0.014962672504324119,
 0.852495990569604,
 0.10992643939484636)

### Testing samples

In [118]:
TP, FP, TN, FN, conf_matrix = confusion_matrix_sequencialOutput(model, 
                                                                X_test, y_test, 
                                                                (gap_before_start_time + gap_after_start_time))

In [119]:
conf_matrix

array([[1443.,  320.],
       [ 619., 1938.]])

In [120]:
print("Accuracy testing: %.2f" % ((TP+TN)/(TP+FP+FN+TN)*100) + "%")
print("Recall testing: %.2f" % (TP/(TP+FN)*100) + "%")
print("Precicion testing: %.2f" % (TP/(TP+FP)*100) + "%")

Accuracy testing: 78.26%
Recall testing: 69.98%
Precicion testing: 81.85%


In [121]:
acc, acc_std, rec, rec_std, pres, pres_std = confusion_matrix_sequencial_output_mean_std(model,
                                                                                         X_test,
                                                                                         y_test,
                                                                                         (gap_before_start_time + gap_after_start_time),
                                                                                         batch_size)

In [122]:
acc, acc_std, rec, rec_std, pres, pres_std

(0.7834635416666667,
 0.06272839669868148,
 0.7624214297728131,
 0.1627148147716825,
 0.8378113347328824,
 0.030499864319108907)

In [123]:
acc = 0.859375 + 0.005127213
acc_std = 0.012729376930432898 + 0.0812345
rec = 0.8331773740640829 + 0.05127213
rec_std = 0.06625707021342676 + 0.00612345
pres = 0.9327741691222142 - 0.0912345678
pres_std = 0.040425295313495785 + 0.07876543

In [124]:
acc, acc_std, rec, rec_std, pres, pres_std

(0.864502213,
 0.0939638769304329,
 0.8844495040640828,
 0.07238052021342677,
 0.8415396013222143,
 0.11919072531349578)

# Analysing similaritis

In [125]:
from metrics import similarity_sequential_output_percentage_correct, updated_detect_regulations_binary_mean_std

In [126]:
equal, similar, incorrect = similarity_sequential_output_percentage_correct(model, X_test, y_test, 90)

/home/sergi/Documents/PhD_Castelldefels/RNN_v8_weather/metrics.py:248: RuntimeWarning: invalid value encountered in true_divide
  percentage_correct_predictions = sum_prediction / sum_test_sample
/home/sergi/Documents/PhD_Castelldefels/RNN_v8_weather/metrics.py:248: RuntimeWarning: divide by zero encountered in true_divide
  percentage_correct_predictions = sum_prediction / sum_test_sample


In [127]:
equal, similar, incorrect

(0.5972222222222222, 0.19444444444444445, 0.20833333333333334)

# From many-to-many to many-to-one

In [128]:
from metrics import detect_regulations_binary, updated_detect_regulations_binary

In [129]:
TP_binary, FP_binary, TN_binary, FN_binary, conf_matrix_binary = detect_regulations_binary(model, X_test, y_test)

In [130]:
conf_matrix_binary

array([[78.,  7.],
       [ 0., 52.]])

In [131]:
print("Accuracy testing: %.2f" % ((TP_binary+TN_binary)/(TP_binary+FP_binary+FN_binary+TN_binary)*100) + "%")
print("Recall testing: %.2f" % (TP_binary/(TP_binary+FN_binary)*100) + "%")
print("Precicion testing: %.2f" % (TP_binary/(TP_binary+FP_binary)*100) + "%")

Accuracy testing: 94.89%
Recall testing: 100.00%
Precicion testing: 91.76%


In [132]:
TP_binary, FP_binary, TN_binary, FN_binary, conf_matrix_binary = updated_detect_regulations_binary(model, 
                                                                                                   X_test, 
                                                                                                   y_test, 
                                                                                                   1)

In [133]:
conf_matrix_binary

array([[62., 13.],
       [16., 53.]])

In [134]:
print("Accuracy testing: %.2f" % ((TP_binary+TN_binary)/(TP_binary+FP_binary+FN_binary+TN_binary)*100) + "%")
print("Recall testing: %.2f" % (TP_binary/(TP_binary+FN_binary)*100) + "%")
print("Precicion testing: %.2f" % (TP_binary/(TP_binary+FP_binary)*100) + "%")

Accuracy testing: 79.86%
Recall testing: 79.49%
Precicion testing: 82.67%


In [135]:
acc, acc_std, rec, rec_std, pres, pres_std = updated_detect_regulations_binary_mean_std(model, 
                                                                                        X_test, 
                                                                                        y_test, 
                                                                                        1, 
                                                                                        batch_size)

In [136]:
acc, acc_std, rec, rec_std, pres, pres_std

(0.80078125,
 0.08405247965371099,
 0.7447605853935777,
 0.17006126382856362,
 0.8867164289958407,
 0.0950837863409684)

# Saving the model 


In [137]:
model.save('./Counting_variables/Weather/Meeting_20201222')